In [65]:
import os
for dirname, _, filenames in os.walk('/kaggel/input'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

In [66]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import warnings
try:
    import apyori
except:
    %pip install apyori
from apyori import apriori

In [67]:
warnings.filterwarnings('ignore')

In [68]:
df= pd.read_csv(r"C:\Users\aujal\OneDrive\Desktop\AI Practial\A.I\Groceries_dataset.csv", parse_dates=['Date'])
df.head()

,Member_number,Date,itemDescription
0,1808,2015-07-21,tropical fruit
1,2552,2015-01-05,whole milk
2,2300,2015-09-19,pip fruit
3,1187,2015-12-12,other vegetables
4,3037,2015-02-01,whole milk


In [69]:
df.isnull().any()

Member_number      False
Date               False
itemDescription    False
dtype: bool

In [70]:
all_products= df['itemDescription'].unique()
print("Total Producst: {}".format(len(all_products)))

Total Producst: 167


In [71]:
def distribution_plot(x,y, name=None, xaxis=None,yaxis=None):
    fig=go.Figure([
        go.Bar(x=x, y=y)
    ])

    fig.update_layout(
        title_text=name,
        xaxis_title=xaxis,
        yaxis_title=yaxis
    )

    fig.show()

In [72]:
x=df['itemDescription'].value_counts()
x=x.sort_values(ascending=False)
x=x[:10]

distribution_plot(x=x.index, y=x.values, yaxis="Count", xaxis="Products")

In [73]:
one_hot = pd.get_dummies(df['itemDescription'])
df.drop('itemDescription', inplace=True, axis=1)
df = df.join(one_hot)
df.head()

,Member_number,Date,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,1808,2015-07-21,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2552,2015-01-05,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,2300,2015-09-19,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1187,2015-12-12,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,3037,2015-02-01,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [74]:
records= df.groupby(["Member_number","Date"])[all_products[:]].apply(sum)
records= records.reset_index()[all_products]

In [75]:
#Replacing non-zero values with product names
def get_Pnames(x):
    for product in all_products:
        if x[product] >0:
            x[product] = product
    return x

records= records.apply(get_Pnames, axis=1)
records.head()

,tropical fruit,whole milk,pip fruit,other vegetables,rolls/buns,pot plants,citrus fruit,beef,frankfurter,chicken,...,flower (seeds),rice,tea,salad dressing,specialty vegetables,pudding powder,ready soups,make up remover,toilet cleaner,preservation products
0,0,whole milk,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,whole milk,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
print("Total Transactions: {}".format(len(records)))

Total Transactions: 14963


In [77]:
x= records.values
x= [sub[~(sub ==0)].tolist() for sub in x if sub[sub !=0].tolist()]
transaction=x

In [78]:
transaction[0:10]

[['whole milk', 'pastry', 'salty snack'],
 ['whole milk', 'yogurt', 'sausage', 'semi-finished bread'],
 ['soda', 'pickled vegetables'],
 ['canned beer', 'misc. beverages'],
 ['sausage', 'hygiene articles'],
 ['whole milk', 'rolls/buns', 'sausage'],
 ['whole milk', 'soda'],
 ['frankfurter', 'soda', 'whipped/sour cream'],
 ['beef', 'white bread'],
 ['frankfurter', 'curd']]

In [79]:
rules = apriori(transaction, min_support=0.00030, min_confidance=0.05, min_list=3,min_length=2,target="rules")
association_results=list(rules)

In [80]:
for item in association_results:
    pair= item[0]
    items= [x for x in pair]

    if len(items) >=2:
        print("Rules: " + items[0] + "->" + items[1])
    else:
        print("Insufficient items for rule.")

    print("Suport: " + str(item[1]))

    print("COnfidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("======================================")

Insufficient items for rule.
Suport: 0.004009891064626078
COnfidence: 0.004009891064626078
Lift: 1.0
Insufficient items for rule.
Suport: 0.021386085678005748
COnfidence: 0.021386085678005748
Lift: 1.0
Insufficient items for rule.
Suport: 0.0014702933903628951
COnfidence: 0.0014702933903628951
Lift: 1.0
Insufficient items for rule.
Suport: 0.0019381140145692708
COnfidence: 0.0019381140145692708
Lift: 1.0
Insufficient items for rule.
Suport: 0.008086613646995923
COnfidence: 0.008086613646995923
Lift: 1.0
Insufficient items for rule.
Suport: 0.0011361358016440553
COnfidence: 0.0011361358016440553
Lift: 1.0
Insufficient items for rule.
Suport: 0.03395041101383412
COnfidence: 0.03395041101383412
Lift: 1.0
Insufficient items for rule.
Suport: 0.021787074784468355
COnfidence: 0.021787074784468355
Lift: 1.0
Insufficient items for rule.
Suport: 0.016574216400454454
COnfidence: 0.016574216400454454
Lift: 1.0
Insufficient items for rule.
Suport: 0.04531176903027468
COnfidence: 0.0453117690302746